## INFX574 Problem Set 4
<br> __Yourname: Xinyu Li__ 
<br> __Deadline: Wed, May16, 5:30__

#### Introduction
<br>This problem set is long but I hope it is still doable :-) It revolves around classi cation, logistic regression and regularization. At the end, you are asked to construc a ROC curve.

<br>Please submit a) your code (notebooks) and b) the results in a  nal output form (html or pdf). I recommend to use scikit-learn.linear_model .

<br> You are welcome to answer some of the questions on paper but please include the result as an image in your  nal  le. Note that you can easily include images in both notebooks and .rmd besides of the code, both are just markdown documents.

<br> Working together is fun and useful but you have to submit your own work. Discussing the solutions and problems with your classmates is all right but do not copy-paste their solution! Please list all your collaborators below:
1. 
2. 


#### Wisconsin Breast Cancer Dataset
<br> You will work with Wisconsin Breast Cancer Dataset (WBCD), available at UCI Machine Learning Repos-itory. You can download it from the internet but rather use the  les wdbc.csv.bz2 and wdbc_doc.txt from canvas (under files/data ) where I have added the variable names to the data. The  rst  le is the csv with variable names, the second one a brief description of the data.

<br> The data includes diagnosis of the tumor with  M  meaning cancer (malignant) and  B  no cancer (benign), and 10 features, describing physical properties of cell nuclei from biopsy samples. Each feature is represented three times, once for mean, once for standard error, and once for the worst values. Your task is to predict diagnosis based on this data.

### 1.  Explore the data 
__1.	Load the data. You may drop id or just ignore it in the rest of your analysis.__

__2. Create a summary table where you show means, ranges, and number of missings for each variable. In addition, add correlation between the diagnosis and the corresponding feature. You may include more statistics you consider useful.__

__3.	Graphical exploration. Make a number of scatterplots where you explore the relationship between features and the diagnosis.__

### 2.	Decision Boundary
<br>The  rst task is to plot the decision boundary by kNN and by logistic regression. You will also play a little with feature engineering.

<br>If you are uncertain what is decision boundary, I recommend to consult James et al. (2015) book Section 2. For instance, Figure 2.13 on p38 depicts a 2D classi cation case where certain X1, X2 values are classified as orange and others as blue. Decision boundary is the dashed winding line that separates these two regions.

<br>There are two broad strategies to plot it. In any case, you have  rst to estimate (train) your model. Thereafter you have to predict the classes (cancer/no cancer here) on a regular dense grid that covers the parameter space (this is the small blue/orange dots on  gure 2.13). Afterwards you can either plot your predicted values with a certain color code, or alternatively, say, set predicted M = 1 and predicted B = 0, and make a contour plot for a single contour at level 0.5. You may also combine these both methods.

<br>We ignore training/testing/over tting issues for now.

#### 2.1	kNN Case
<br>First, let's explore the decision boundary using kNN.
<br>Pick two features. I recommend to use a few that show relative strong correlation with diagnosis. Feel free to use a combination you already plotted above.

__ 1.	Predict the diagnosis on a grid (say, 100x100) that covers the range of the explanatory variables. Use kNN with k = 3..7 (pick just one value). This gives you 100x100 predicted diagnoses.__

Note: if your features are of very di erent scale, you should either scale these into a roughly equal scale, or use a metric that does this with you. Consult James et al. (2015, p 217).

__2.	Plot the actual data and the decision boundary on the same plot. Ensure that actual observations and predictions are clearly distinguishable, and that one can easily understand the color code.__

__3.	Describe your observations. How good is kNN in picking up the actual shape? Does it also pick up noise?__

Note: unless you do cross-validation, you cannot know if the model picks up noise (over t). Here I just ask your best judgement, not any formal analysis.

#### 2.2	Logistic Regression
Now repeat the process above by logistic regression. Pick the same features as what you used for k-NN above.
__1.	Fit a logistic regression model with these two features.__

__2.	Predict the diagnosis on a similar grid. . .__

__3. . . . and create a similar plot.__

__4. Describe your observations. How does the result for kNN compare to that for Logistic Regression?__

#### 2.3	Feature Engineering
So far you were using just two of the existing features in the data. However, now let's create a few more.
<br>__1.	Use these two features to compute some new ones. Let's denote your original features by x and y.Examples you may create include: x2, y2, x y, 1(x > 5), 1(y < 1) x2, log x . . . You can use all sorts of mathematical operations as long as a) you only use x and y, not other features, and b) the original and the engineered features remain linearly independent (they are, unless you create features like x +    y).__

__2.	Fit a logistic regression model. However, this time pick both x, y, and some of your engineered features.__

__3.	Create the decision boundary plot.__

__4.	Comment on the shape of the boundary. What do you think, how well can you capture the actual boundary? What about over tting? (Again, I ask about your judgement, not about any formal analysis).__

__5.	Repeat the exercise a few times where you pick/engineer di erent new features, and try to get as reasonable boundary as you can.__

<br>As above, I am asking  reasonable  boundary in the sense of your best judgement. No actual cross validation is necessary.

<br>Note: I have mixed experience with scikit-learn.linear_model.LogisticRegression . It occasionally appears the default convergence tolerance is far too big, and the default liblinear solver too imprecise. Setting tolerance to 10-12 and solver to lbfgs improved the results for me, but did not make it  awless.

### 3	Use the full data
<br>Finally, we'll get serious. We'll use full data set in the logistic regression, include regularization, and cross-validate our results.

#### 3.1	Cross Validation
<br>Here your task is to Write code to do n-fold cross validation. Note: I expect you to implement CV here, not to use any canned version. Consult James et al. (2015, Section 5.1.3).

__ 1.	You should do k-fold CV (k > 10) with the following tasks:
<br>(a)	Fit logistic regression model on training data using all existing features. You may also add your engineered features if you wish.
<br>(b)	Calculate accuracy, precision and recall on the validation data.__

__2.	Report the average accuracy, precision, and recall over your CV runs.__

#### 3.2	Regularization
<br>Your (almost) last task is to  nd the best regularization parameter using CV algorithm you created above. Consult James et al. (2015, Section 6.2) for introduction to regularization methods (ridge and lasso). I expect you to play with regularization parameters in existing software, such as scikit-learn.linear_model , not to implement it yourself.

__ 1. Pick a type of regularization (lasso, ridge, elastic net).__

__2.	Create a wide list of regularization parameters. And I mean wide, for instance ranging from 10-6 to 106. Pick a number of values inside this range.__

__3.	For each in this range, repeat the CV process in exercise 3.1 above. Compute accuracy, precision, recall.__
<br>Warning: this may be slow, so I recommend you to start with only a few values. When you algorithm works well, increase the number of values.

__4.	Report the results as a function of  . This may be in the form of a table or a graph.__

__5.	Report the best regularization parameters, and the best results.__

### 4	ROC curve: which estimator is the best
<br>Your last task is to create the ROC curve of several estimators. Consult James et al. (2015, p 148) for what is the ROC curve. You have to compare a) k-NN, b) Naive Bayes, and c) logistic regression estimators.

<br> In case of k-NN, you should pick a few di erent k values (say, 1, 5 and 25). In case of Naive Bayes, you may use use sklearn.naive_bayes.GaussianNB . Feel free to use it  as is , no calibration necessary.

__1.	Split your data into testing and training sets (you may use sklearn.model_selection.train_test_split ), say 1/3 for testing. Use all the features in the original data.__

__2.	Using your training data, estimate (a few) k-NN models, Naive Bayes model, and logistic regression.__
<br>Notes: in case of k-NN, you should use scaled features. In case of logistic regression, use the optimal regularization parameter value you found in 3.2.

__3.	For each of the models, predict the probabilites on the test data.__
<br>Note: you have to use predict_proba, not predict for logistic and NB regression.

__4.	Pick a number of thresholds between 0 and 1 (for instance, 0, 0.1, 0.2, ...). For each model and each threshold, treat your predictions to be 1 if it it's probability is at least as big as the threshold.__

__5.	Based on these predictions, compute false positive rate and true positive rate.__

__6.	Plot these rates for each threshold and model.__

__7.	Comment your results. Which model is the best?__